In [2]:
# Computational Intelligence - Lab 1
#
# For large-scale problems like Problem 3, fitness function,
# which recalculated the entire solution's value and penalty from scratch
# in every step, was a major bottleneck.
#
# The main loop uses an "incremental update" or "delta evaluation".
# Since each move only involves a single item, I can calculate the change in fitness 
# through all 5,000 items. This makes each step of the algorithm
# orders of magnitude faster and is a critical optimization for
# tackling large problem instances in a reasonable amount of time.
#

import numpy as np
import time
import math
from tqdm import trange

# --- Reusable Solver with Optimized Fitness Calculation ---

def solve_knapsack_problem(problem_name, values, weights, constraints, steps=20000, initial_temp=100.0, cooling_rate=0.9995):
    """
    An optimized Simulated Annealing solver that uses incremental
    fitness updates for much faster execution on large problems.
    """
    
    num_items = len(values)
    num_knapsacks = len(constraints)
    penalty_multiplier = np.sum(values)
    rng = np.random.default_rng(seed=42)

    # --- Full Fitness Calculation (used only for initialization) ---
    def calculate_initial_fitness(solution):
        total_value = 0
        knapsack_weights = np.zeros_like(constraints, dtype=float)
        for i, k in enumerate(solution):
            if k != -1:
                total_value += values[i]
                knapsack_weights[k] += weights[i]
        overload = np.maximum(0, knapsack_weights - constraints)
        return total_value - np.sum(overload) * penalty_multiplier

    # --- Algorithm Initialization ---
    current_solution = np.full(num_items, -1, dtype=int)
    for item_idx in rng.permutation(num_items):
        k = rng.integers(num_knapsacks)
        temp_sol = current_solution.copy()
        temp_sol[item_idx] = k
        # Use full calculation only for this initial greedy setup
        if calculate_initial_fitness(temp_sol) > calculate_initial_fitness(current_solution):
             current_solution = temp_sol

    # Calculate initial state once using the full method
    current_knapsack_weights = np.zeros_like(constraints, dtype=float)
    current_value = 0
    for i, k in enumerate(current_solution):
        if k != -1:
            current_value += values[i]
            current_knapsack_weights[k] += weights[i]
    
    overload = np.maximum(0, current_knapsack_weights - constraints)
    current_fitness = current_value - np.sum(overload) * penalty_multiplier

    best_solution = current_solution.copy()
    best_fitness = current_fitness
    temperature = initial_temp
    
    print(f"\n--- Solving {problem_name} (Optimized) ---")
    print(f"Starting SA. Initial fitness: {current_fitness:.2f}")

    # --- Main Optimized SA Loop ---
    for step in trange(steps, desc=f"Annealing {problem_name}"):
        item_to_tweak = rng.integers(0, num_items)
        old_assignment = current_solution[item_to_tweak]
        new_assignment = rng.integers(-1, num_knapsacks)

        if old_assignment == new_assignment:
            continue
        
        # *** OPTIMIZATION: Calculate fitness delta ***
        new_knapsack_weights = current_knapsack_weights.copy()
        new_value = current_value
        
        # Step 1: Revert the old assignment
        if old_assignment != -1:
            new_value -= values[item_to_tweak]
            new_knapsack_weights[old_assignment] -= weights[item_to_tweak]
        
        # Step 2: Apply the new assignment
        if new_assignment != -1:
            new_value += values[item_to_tweak]
            new_knapsack_weights[new_assignment] += weights[item_to_tweak]
            
        new_overload = np.maximum(0, new_knapsack_weights - constraints)
        new_fitness = new_value - np.sum(new_overload) * penalty_multiplier
        
        fitness_delta = new_fitness - current_fitness
        
        # Acceptance check
        if fitness_delta > 0 or (temperature > 0 and rng.random() < math.exp(fitness_delta / temperature)):
            # If accepted, update all current state variables
            current_solution[item_to_tweak] = new_assignment
            current_knapsack_weights = new_knapsack_weights
            current_value = new_value
            current_fitness = new_fitness

        if current_fitness > best_fitness:
            best_solution = current_solution.copy()
            best_fitness = current_fitness
        
        temperature *= cooling_rate
        if temperature < 1e-3:
            break

    # --- Reporting Results ---
    final_total_value = 0
    for item_idx, knapsack_idx in enumerate(best_solution):
        if knapsack_idx != -1:
            final_total_value += values[item_idx]

    print(f"\n--- Results for {problem_name} ---")
    print(f"Best solution found has value: {final_total_value}")
    print("Final solution is feasible.") # Implicitly checked by fitness
    print("="*40)
    return final_total_value, best_solution

# --- PROBLEM DEFINITIONS AND EXECUTION ---

if __name__ == "__main__":
    
    # --- Problem 1 Data ---
    rng_p1 = np.random.default_rng(seed=42)
    P1_VALUES = rng_p1.integers(0, 100, size=20)
    P1_WEIGHTS = rng_p1.integers(0, 100, size=(20, 2))
    P1_CONSTRAINTS = rng_p1.integers(0, 100 * 20 // 3, size=(3, 2))

    # --- Problem 2 Data ---
    rng_p2 = np.random.default_rng(seed=42)
    P2_VALUES = rng_p2.integers(0, 1000, size=100)
    P2_WEIGHTS = rng_p2.integers(0, 1000, size=(100, 10))
    P2_CONSTRAINTS = rng_p2.integers(1000 * 2, 1000 * 100 // 10, size=(10, 10))

    # --- Problem 3 Data ---
    rng_p3 = np.random.default_rng(seed=42)
    P3_VALUES = rng_p3.integers(0, 1000, size=5000)
    P3_WEIGHTS = rng_p3.integers(0, 1000, size=(5000, 100))
    P3_CONSTRAINTS = rng_p3.integers(1000 * 10, 1000 * 2 * 5000 // 100, size=(100, 100))

    # --- Run Solvers ---
    start_time_all = time.time()
    
    solve_knapsack_problem(
        "Problem 1", P1_VALUES, P1_WEIGHTS, P1_CONSTRAINTS,
        steps=20000, initial_temp=100, cooling_rate=0.9995
    )
    
    solve_knapsack_problem(
        "Problem 2", P2_VALUES, P2_WEIGHTS, P2_CONSTRAINTS,
        steps=100000, initial_temp=1000, cooling_rate=0.9999
    )
    
    solve_knapsack_problem(
        "Problem 3", P3_VALUES, P3_WEIGHTS, P3_CONSTRAINTS,
        steps=1000000, initial_temp=1000, cooling_rate=0.99999
    )

    end_time_all = time.time()
    print(f"\nTotal execution time for all problems: {end_time_all - start_time_all:.2f} seconds.")


--- Solving Problem 1 (Optimized) ---
Starting SA. Initial fitness: 792.00


Annealing Problem 1: 100%|████████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 19379.20it/s]



--- Results for Problem 1 ---
Best solution found has value: 1065
Final solution is feasible.

--- Solving Problem 2 (Optimized) ---
Starting SA. Initial fitness: 29964.00


Annealing Problem 2: 100%|██████████████████████████████████████████████████| 100000/100000 [00:06<00:00, 14726.05it/s]



--- Results for Problem 2 ---
Best solution found has value: 47835
Final solution is feasible.

--- Solving Problem 3 (Optimized) ---
Starting SA. Initial fitness: 1109340.00


Annealing Problem 3: 100%|█████████████████████████████████████████████████| 1000000/1000000 [02:11<00:00, 7627.27it/s]


--- Results for Problem 3 ---
Best solution found has value: 1565519
Final solution is feasible.

Total execution time for all problems: 250.34 seconds.
